<center>
  <font size="4">CIFAR100 Image Classification</font><br>
  <font size="3">  -- Milestone 3</font><br>
  Xiyu Cao,	Meng Wang, Yuxue Jiang, Mengguo Yan, Qianhui Lu, Yihui Zhou</center>



In [0]:

from __future__ import print_function
import tensorflow as tf
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation

from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.utils import to_categorical
from keras.models import load_model
from keras.optimizers import SGD
from keras.regularizers import l1, l2
from keras.utils import np_utils

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *
import warnings
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import time
from sklearn.decomposition import PCA
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


### Data preparation
1) Data first loaded from external site and label preparation

In [0]:
def PreprocessDataset():
    (x_train_org, y_train_org), (x_test_org, y_test_org) = cifar100.load_data(label_mode='fine')
    x_train_org = x_train_org.astype('float32')
    x_test_org = x_test_org.astype('float32')
    # Normalize value to [0, 1]
    x_train_org /= 255
    x_test_org /= 255
    # Transform lables to one-hot
    y_train_org = np_utils.to_categorical(y_train_org, 100)
    y_test_org = np_utils.to_categorical(y_test_org, 100)
    # Reshape: here x_train is re-shaped to [channel] × [width] × [height]
    # In other environment, the orders could be different; e.g., [height] × [width] × [channel].
    x_train_org = x_train_org.reshape(x_train_org.shape[0], 32, 32, 3)
    x_test_org = x_test_org.reshape(x_test_org.shape[0], 32, 32, 3)
    return [x_train_org, x_test_org, y_train_org, y_test_org]
x_train_org, x_test_org, y_train_org, y_test_org = PreprocessDataset()

CIFAR100_LABELS_LIST = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

#Label chose:
label_choosen = ['bridge', 'castle', 'house', 'road', 'skyscraper','cloud', 'forest', 'mountain', 'plain','sea']
label_loc = []
for item in CIFAR100_LABELS_LIST:
  if item in label_choosen:
    print([CIFAR100_LABELS_LIST.index(item), item])
    label_loc.append(CIFAR100_LABELS_LIST.index(item))

[12, 'bridge']
[17, 'castle']
[23, 'cloud']
[33, 'forest']
[37, 'house']
[49, 'mountain']
[60, 'plain']
[68, 'road']
[71, 'sea']
[76, 'skyscraper']


2) Combine testing and training data into one after you load the cifar100 data


In [0]:
def concat(train,test):
  res = np.concatenate((train,test), axis = 0)
  print ('Shape:', res.shape)
  return res
x = concat(x_train_org,x_test_org)
y = concat(y_train_org,y_test_org)

Shape: (60000, 32, 32, 3)
Shape: (60000, 100)


3) Filter to select images belonging to your assigned superclasses and make it ready to use

In [0]:
#set up subclass
def subclass(x, y):
  x_dict = {}
  y_dict = {}
  for i, item in enumerate(y):
    if np.argmax(item) in label_loc:
      if np.argmax(item) in x_dict.keys():
        x_dict[np.argmax(item)].append(x[i])
        y_dict[np.argmax(item)].append(y[i])
      else:
        x_dict[np.argmax(item)] = []
        x_dict[np.argmax(item)].append(x[i])
        y_dict[np.argmax(item)] = []
        y_dict[np.argmax(item)].append(y[i])
  return x_dict, y_dict

x_dict,y_dict = subclass(x, y)
#x_train_dict, y_train_dict = subclass(x_train_org, y_train_org)
#x_test_dict, y_test_dict = subclass(x_test_org, y_test_org)

# return np.array
binary_label_loc_0 = [12,17,37,68,76]
#binary_label_loc_test_0 = [37]
binary_label_loc_1 = [23,33,49,60,71]
#binary_label_loc_test_1 = [60]

def subclass_np(x_dict, y_dict, labels):
  x = []
  y= []
  for key in labels:
    x.append(x_dict[key])
    y.append(y_dict[key])
  x = np.array(x)
  x = x.reshape(x.shape[0]*x.shape[1], 32,32,3)
  y = np.array(y)
  y =  y.reshape(y.shape[0]*y.shape[1], 100,1)
  return x,y

x_0, y_0 = subclass_np(x_dict,y_dict,binary_label_loc_0)
#x_test_0,y_test_0 =    subclass_np(x_dict,y_dict,binary_label_loc_test_0)
x_1, y_1 = subclass_np(x_dict,y_dict, binary_label_loc_1)
#x_test_1, y_test_1 =   subclass_np(x_dict,y_dict, binary_label_loc_test_1)

In [0]:
#x_train = np.concatenate((x_train_0,x_train_1), axis = 0)
#x_test = np.concatenate((x_test_0,x_test_1),axis = 0)
#y_train_0 = np.zeros(x_train_0.shape[0])
#y_train_1 = np.ones(x_train_1.shape[0])
#y_test_0 = np.zeros(x_test_0.shape[0])
#y_test_1 = np.ones(x_test_1.shape[0])

#x = np.concatenate((x_0,x_1), axis = 0)


y_0 = np.zeros(x_0.shape[0])
y_1 = np.ones(x_1.shape[0])
y = np.concatenate((y_0,y_1), axis = 0)

#x = x.reshape(x.shape[0], 32*32*3)
x_0.shape

(3000, 32, 32, 3)

Image printing function definition 

In [0]:
#Print img
def img_print(y_test, y_test_pred, model):
  index = [np.random.choice(range(len(y_test))) for i in range(6)]
  fig = plt.figure(figsize=(5, 5))
  fig.suptitle('Randomly selected pictures from %s'%model)
  n = 0
  for j in range(3):
    for k in range(2):
      i_inds = (j*2)+k
      i_data = index[n]
      ax = fig.add_subplot(2,3, i_inds+1)
      ax.imshow(x_test[i_data].reshape(32,32,3).astype(float), interpolation='nearest')
      label = "ManMade" if y_test_pred[i_data] == 0 else "Natural"
      if y_test[i_data] != y_test_pred[i_data]:
        ax.set_title(label, color = 'red')
      else:
        ax.set_title(label)  
      plt.axis('off')
      n+=1
  fig.tight_layout(rect=[0, 0.03, 1, 0.95])

### Data Selection
1. Based on previous milestone, the best performance include [Plain, House] and the worst include [Forest, Bridge]
2. So we choose three combinations of data for milestone3 as followed: <br>
      a. Test data: [House, Skyscraper] vs [Plain, Sea]  (37, 76) vs (60,71)   <br>
      b. Test data: [Bridge, Road] vs  [Forest, Mountain]  (12, 68) vs (33, 49)<br>
      c. Test data: [House, Road] vs [Forest, Plain]  (37, 68) vs  (33, 60)<br>


In [0]:
def data_selection(label_test_0, label_test_1, x_dict):
  label_0 = [12,17,37,68,76]
  label_1 = [23,33,49,60,71]
  x_train_0 = []
  x_test_0 = []
  x_train_1 = []
  x_test_1 = []
  #print(label_test_0)
  for label in x_dict.keys():
    if label in label_0:
      if label in label_test_0:
          x_test_0.append(x_dict[label])
      else:
        x_train_0.append(x_dict[label])
    if label in label_1:
      if label in label_test_1:
        x_test_1.append(x_dict[label])
      else:
        x_train_1.append(x_dict[label])
  #print([len(x_train_0), len(x_test_0)])
  x_train = np.concatenate((x_train_0, x_train_1), axis = 0)
  x_train = x_train.reshape(6*600,32,32,3)
  x_test = np.concatenate((x_test_0, x_test_1), axis = 0)
  x_test = x_test.reshape(4*600, 32*32*3)
  return x_train, x_test

In [0]:
y_train = np.concatenate((np.zeros(1800), np.ones(1800)), axis = 0)
y_test = np.concatenate((np.zeros(1200), np.ones(1200)), axis = 0)

In [0]:
from keras.preprocessing.image import ImageDataGenerator
def data_augmentation(x_0, y_0, augment_size=14400): 
        
        train_size = x_0.shape[0]
        #train_1 = x_1.shape[0]
        image_generator = ImageDataGenerator(
            rescale = 1./255.,
            featurewise_center=True,
            featurewise_std_normalization=True,
            rotation_range=20,
            zoom_range = 0.05, 
            width_shift_range=0.07,
            height_shift_range=0.07,
            horizontal_flip=True,
            vertical_flip=False, 
            data_format="channels_last",
            zca_whitening=True)
        # fit data for zca whitening
        image_generator.fit(x_0, augment=True)
        # get transformed images
        randidx = np.random.randint(train_size, size=augment_size)
        x_augmented = x_0[randidx].copy()
        y_augmented = y_0[randidx].copy()
        x_augmented = image_generator.flow(x_augmented, np.zeros(augment_size),
                                    batch_size=augment_size, shuffle=False).next()[0]
        # append augmented data to trainset
        x_0 = np.concatenate((x_0, x_augmented))
        y_0 = np.concatenate((y_0, y_augmented))
        #train_size = x_0.shape[0]
        #test_size = x_test.shape[0]
        
        return x_0, y_0

In [0]:
import cv2
def color_pix_r(x):
    percentage = []
    for img in x:
    #color = set()
        pix = img.reshape(1024,3)
        clr = [str(p) for p in pix]
        color = list(set(clr))
        per = len(color)/1024
        percentage.append(per)
    percentage = np.array(percentage)
    return percentage

def color_sat(x):
    sat_array = []
    for array in x:
        array = array.reshape(1024,3)
        sat = np.array([max(abs(pix[0]-pix[1]),abs(pix[0]-pix[2]),abs(pix[2]-pix[1])) for pix in array])
        sat_array.append(np.mean(sat))
    return np.array(sat_array)
#----------------------------------------------------
def geo_trans(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(float)
    rows,cols = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2),90,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    return dst.flatten()

def transform_geo(x_train, x_test):
    x_train_transformed = []
    for item in x_train:
        x_train_transformed.append(geo_trans(item))
    x_test_transformed = []
    for item in x_test:
        x_test_transformed.append(geo_trans(item))
    return np.array(x_train_transformed), np.array(x_test_transformed)
#----------------------------------------------------
def img_gradient(image):
    img = cv2.cvtColor(image.reshape(32,32,3), cv2.COLOR_BGR2GRAY).astype(float)
    laplacian = cv2.Laplacian(img,cv2.CV_64F)
    sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
    sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
    return np.array([laplacian, sobelx, sobely]).flatten()

def transform_gradient(x_train, x_test):
    x_train_transformed = []
    for i, item in enumerate(x_train):
        x_train_transformed.append(img_gradient(item))
        x_test_transformed = []
    for i, item in enumerate(x_test):
        x_test_transformed.append(img_gradient(item))
    return np.array(x_train_transformed), np.array(x_test_transformed)
#----------------------------------------------------
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image.reshape(32,32,3), cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [32, 32, 32], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    return hist.flatten()
  
def transform_hist(x_train, x_test):
    x_train_transformed = []
    for item in x_train:
        x_train_transformed.append(fd_histogram(item))
        x_test_transformed = []
    for item in x_test:
        x_test_transformed.append(fd_histogram(item))
    return np.array(x_train_transformed), np.array(x_test_transformed)
#-------------------------------------------------------
def to_gray(image):
    image = cv2.cvtColor(image.reshape(32,32,3), cv2.COLOR_BGR2GRAY).astype(float)
    return image.reshape(1,image.shape[0]*image.shape[1])[0]


In [0]:
binary_label_loc_0 = [12,17,37,68,76]
binary_label_loc_1 = [23,33,49,60,71]

C_0 = []
for i, first in enumerate(binary_label_loc_0):
    for j, second in enumerate(binary_label_loc_0[i+1:]):
        C_0.append([first, second])
C_1 = []
for i, first in enumerate(binary_label_loc_1):
    for j, second in enumerate(binary_label_loc_1[i+1:]):
        C_1.append([first, second])
        
C_0_name = [[CIFAR100_LABELS_LIST[x[0]],CIFAR100_LABELS_LIST[x[1]]] for x in C_0]
C_1_name = [[CIFAR100_LABELS_LIST[x[0]],CIFAR100_LABELS_LIST[x[1]]] for x in C_1]

###Logistic Regression

In [0]:
result = []
for first_item in C_0:
    for second_item in C_1:
        x_train, x_test = data_selection(first_item,second_item,x_dict )
        x_augment, y_augment = data_augmentation(x_train,y_train, augment_size=9600)
        x_aug_gray = np.array([to_gray(x) for x in x_augment])
        x_test_gray = np.array([to_gray(x) for x in x_test])

        x_aug_sat = color_sat(x_augment)
        x_aug_sat_test = color_sat(x_test)
        x_aug_sat = x_aug_sat.reshape(-1,1)
        x_aug_sat_test = x_aug_sat_test.reshape(-1,1)


        x_aug_grad, x_test_grad =  transform_gradient(x_augment, x_test)
        
        x_augment = x_augment.reshape(x_augment.shape[0], 32*32*3)
        x_aug_combine= np.concatenate((x_aug_gray, x_aug_grad, x_aug_sat), axis = 1)

        x_test = x_test.reshape(x_test.shape[0], 32*32*3)
        x_test_combine = np.concatenate((x_test_gray, x_test_grad, x_aug_sat_test), axis = 1)
        
        pca = PCA(0.9)
        pca.fit(x_aug_combine)
        x_train_aug = pca.transform(x_aug_combine)
        x_test_aug = pca.transform(x_test_combine)
        ### Modify this part ######
        lg = LogisticRegression()
        lg.fit(x_train_aug, y_augment)
        score = lg.score(x_test_aug, y_test)
        #print(score)
        result.append(score)

In [0]:
result_reshape = np.array(result).reshape(10,10)
sbn.heatmap(result_reshape,cmap="Blues")
plt.xticks(np.arange(10), C_0_name, rotation=90)
plt.yticks(np.arange(10), C_1_name, rotation = 45)

### GradientBoosting

In [0]:
result = []
for first_item in C_0:
    for second_item in C_1:
        x_train, x_test = data_selection(first_item,second_item,x_dict )
        x_augment, y_augment = data_augmentation(x_train,y_train, augment_size=9600)
        x_aug_gray = np.array([to_gray(x) for x in x_augment])
        x_test_gray = np.array([to_gray(x) for x in x_test])

        x_aug_sat = color_sat(x_augment)
        x_aug_sat_test = color_sat(x_test)
        x_aug_sat = x_aug_sat.reshape(-1,1)
        x_aug_sat_test = x_aug_sat_test.reshape(-1,1)


        x_aug_grad, x_test_grad =  transform_gradient(x_augment, x_test)
        
        x_augment = x_augment.reshape(x_augment.shape[0], 32*32*3)
        x_aug_combine= np.concatenate((x_aug_gray, x_aug_grad, x_aug_sat), axis = 1)

        x_test = x_test.reshape(x_test.shape[0], 32*32*3)
        x_test_combine = np.concatenate((x_test_gray, x_test_grad, x_aug_sat_test), axis = 1)
        
        pca = PCA(0.9)
        pca.fit(x_aug_combine)
        x_train_aug = pca.transform(x_aug_combine)
        x_test_aug = pca.transform(x_test_combine)
        ### Modify this part ######
        clf = GradientBoostingClassifier()
        clf.fit(x_train_aug, y_augment)
        score = clf.score(x_test_aug, y_test)
        #print(score)
        result.append(score)

In [0]:
result_reshape = np.array(result).reshape(10,10)

In [0]:
sbn.heatmap(result_reshape,cmap="Blues")
plt.xticks(np.arange(10), C_0_name, rotation=90)
plt.yticks(np.arange(10), C_1_name, rotation = 45)

### Random Forest

In [0]:
result = []
for first_item in C_0:
    for second_item in C_1:
        x_train, x_test = data_selection(first_item,second_item,x_dict )
        x_augment, y_augment = data_augmentation(x_train,y_train, augment_size=9600)
        x_aug_gray = np.array([to_gray(x) for x in x_augment])
        x_test_gray = np.array([to_gray(x) for x in x_test])

        x_aug_sat = color_sat(x_augment)
        x_aug_sat_test = color_sat(x_test)
        x_aug_sat = x_aug_sat.reshape(-1,1)
        x_aug_sat_test = x_aug_sat_test.reshape(-1,1)


        x_aug_grad, x_test_grad =  transform_gradient(x_augment, x_test)
        
        x_augment = x_augment.reshape(x_augment.shape[0], 32*32*3)
        x_aug_combine= np.concatenate((x_aug_gray, x_aug_grad, x_aug_sat), axis = 1)

        x_test = x_test.reshape(x_test.shape[0], 32*32*3)
        x_test_combine = np.concatenate((x_test_gray, x_test_grad, x_aug_sat_test), axis = 1)
        
        pca = PCA(0.9)
        pca.fit(x_aug_combine)
        x_train_aug = pca.transform(x_aug_combine)
        x_test_aug = pca.transform(x_test_combine)
        ### Modify this part ######
        clf = GradientBoostingClassifier()
        clf.fit(x_train_aug, y_augment)
        score = clf.score(x_test_aug, y_test)
        #print(score)
        result.append(score)

In [0]:
result_reshape = np.array(result).reshape(10,10)
sbn.heatmap(result_reshape,cmap="Blues")
plt.xticks(np.arange(10), C_0_name, rotation=90)
plt.yticks(np.arange(10), C_1_name, rotation = 45)

### Dense Net


In [0]:
from keras.datasets import cifar100
from __future__ import print_function

import keras.callbacks as cb
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.regularizers import l1, l2
from keras.utils import np_utils

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import time

from sklearn.metrics import classification_report
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

num_predictions = 36
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.applications.densenet import DenseNet201,preprocess_input
import itertools

CIFAR100_LABELS_LIST = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

CIFAR100_LABELS_LIST.sort()


batch_size=64
datagen = ImageDataGenerator(
	featurewise_center=False, 
	samplewise_center=False,  
	featurewise_std_normalization=False,
	samplewise_std_normalization=False, 
	zca_whitening=False, 
	rotation_range=0, 
	width_shift_range=0, 
	height_shift_range=0,
	horizontal_flip=False, 
	vertical_flip=False) 


# https://keras.io/applications/

In [0]:
def PreprocessDataset():
    (x_train_org, y_train_org), (x_test_org, y_test_org) = cifar100.load_data(label_mode='fine')
    x_train_org = x_train_org.astype('float32')
    x_test_org = x_test_org.astype('float32')
    # Normalize value to [0, 1]
    #x_train_org /= 255
    #x_test_org /= 255
    # Transform lables to one-hot
    y_train_org = np_utils.to_categorical(y_train_org, 100)
    y_test_org = np_utils.to_categorical(y_test_org, 100)
    # Reshape: here x_train is re-shaped to [channel] × [width] × [height]
    # In other environment, the orders could be different; e.g., [height] × [width] × [channel].
    x_train_org = x_train_org.reshape(x_train_org.shape[0], 32, 32, 3)
    x_test_org = x_test_org.reshape(x_test_org.shape[0], 32, 32, 3)
    return [x_train_org, x_test_org, y_train_org, y_test_org]

x_train_org, x_test_org, y_train_org, y_test_org = PreprocessDataset()

In [0]:
def subclass(x, y,loc):
  x_dict = {}
  y_dict = {}
  for i, item in enumerate(y):
    if np.argmax(item) in loc:
      if np.argmax(item) in x_dict.keys():
        x_dict[np.argmax(item)].append(x[i])
        y_dict[np.argmax(item)].append(y[i])
      else:
        x_dict[np.argmax(item)] = []
        x_dict[np.argmax(item)].append(x[i])
        y_dict[np.argmax(item)] = []
        y_dict[np.argmax(item)].append(y[i])
  return x_dict, y_dict


# return np.array
def subclass_np(x_dict, y_dict):
  x = []
  y= []
  for key in x_dict.keys():
    x.append(x_dict[key])
    y.append(y_dict[key])
  x = np.array(x,dtype=np.float32)
  x = x.reshape(x.shape[0]*x.shape[1], 32,32,3)
  y = np.array(y)
  y =  y.reshape(y.shape[0]*y.shape[1], 100,1)
  return x,y

binary_label_loc_0 = [12,17,37,68,76]
binary_label_0 = ['bridge', 'castle', 'house', 'road', 'skyscraper']

def y_binary(y_list):
  y_bi = []
  for item in y_list:
    if np.argmax(item) in binary_label_loc_0:
      y_bi.append(0)
    else:
      y_bi.append(1)
  return np.array(y_bi)

def cifar_grid_c(X,Y,inds,n_col, predictions=None):
  if predictions is not None:
    if Y.shape != predictions.shape:
      print("Predictions must equal Y in length!")
      print(Y.shape)
      print(predictions.shape)
      return(None)
  N = len(inds)
  n_row = int(np.ceil(1.0*N/n_col))
  fig, axes = plt.subplots(n_row,n_col,figsize=(10,30))
  
#  clabels = labels["label_names"]
  clabels = ["man-made","nature"]
  for j in range(n_row):
    for k in range(n_col):
      i_inds = (j*n_col)+k
      i_data = inds[i_inds]
      
     # axes[j][k].set_axis_off()
      if i_inds < N:
        rgb = X[[i_data,...]]
        img = rgb.astype(np.uint8)
        #do not need reshap here. img = rgb.reshape(3,32,32).transpose([1, 2, 0])
        axes[j][k].imshow(img)
        label = clabels[np.argmax(Y[i_data,...])]
        axes[j][k].set_title(label)
        if predictions is not None:
          pred = clabels[np.argmax(predictions[i_data,...])]
          if label != pred:
            label += " n"
            axes[j][k].set_title(pred, color='red')            
  
  fig.set_tight_layout(True)
  

In [0]:
import gc 
# will be total of 100 combos
counter = 0
for ab in itertools.combinations(['bridge', 'castle', 'road', 'skyscraper','house'], 2):
  for cd in itertools.combinations(['plain','cloud', 'forest', 'mountain','sea'], 2):
    label_choosen_train = ['bridge', 'castle', 'road', 'skyscraper','house', 'plain', 'cloud', 'forest', 'mountain','sea']
    a,b = ab
    c,d = cd
    label_choosen_test = [a,b,c,d]
    print("#######################################################")
    print("#######################################################")
    print("#######################################################")
    print("####### combo:%d, using %s,%s,%s,%s as testing ########" %(counter,a,b,c,d))
    print("#######################################################")
    print("#######################################################")
    print("#######################################################")
    gc.collect()
    if counter <97:
      counter += 1
      continue
    counter += 1
    for i in label_choosen_test:
      label_choosen_train.remove(i)

    label_loc_train = []
    label_loc_test = []
    for item in CIFAR100_LABELS_LIST:
      if item in label_choosen_train:
        print([CIFAR100_LABELS_LIST.index(item), item])
        label_loc_train.append(CIFAR100_LABELS_LIST.index(item))
      if item in label_choosen_test:
        print([CIFAR100_LABELS_LIST.index(item), item])
        label_loc_test.append(CIFAR100_LABELS_LIST.index(item))

    # sub_class selection -- return dictionary (for later milestone 2 usage)

    x_train_dict, y_train_dict = subclass(x_train_org, y_train_org, label_loc_train)
    x_test_dict, y_test_dict = subclass(x_test_org, y_test_org, label_loc_test)

    x_train, y_train_list = subclass_np(x_train_dict, y_train_dict)
    x_test,  y_test_list = subclass_np(x_test_dict, y_test_dict)
    y_train = y_binary(y_train_list)
    y_test = y_binary(y_test_list)
    print([x_train.shape, y_train.shape, x_test.shape, y_test.shape])
    print([np.unique(y_train),np.bincount(y_train)])
    n = 150
    some_key = list(x_train_dict.keys())[0]
    for i in range(5):
      n = n+1
      plt.subplot(n)
      plt.imshow(x_train_dict[some_key][i])
      plt.imshow(x_train_dict[some_key][i].astype(np.uint8))

    base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=(32,32,3))
    densenet_input_train = preprocess_input(x_train)
    densenet_input_test = preprocess_input(x_test)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(2, activation='softmax')(x)
    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(densenet_input_train, to_categorical(y_train, num_classes=2), batch_size=64, epochs=3,
              validation_data=(densenet_input_test, to_categorical(y_test, num_classes=2)),
              verbose=2, shuffle=True)


    y_tr_predict = model.predict(densenet_input_train)
    y_te_predict = model.predict(densenet_input_test)
    y_predict = np.concatenate((y_tr_predict,y_te_predict))
    y_ = np.concatenate((y_train,y_test))
    print("combined classification report")
    y_predict = np.argmax(y_predict,1)
    print(classification_report(y_, y_predict))
    labels = np.unique(y_)
    cm = confusion_matrix(y_, y_predict,labels)
    plt.figure(figsize = (6,6))
    df_cm = pd.DataFrame(cm, index = [i for i in labels],
                      columns = [i for i in labels])
    ax=sn.heatmap(df_cm, annot=True)
    #print out randomly selected images(black/red labl-->correct/wrong prediction )


    x_ = np.concatenate((x_train,x_test))
    datagen.fit(x_train)
    predict_gen = model.predict_generator(datagen.flow(densenet_input_test, to_categorical(y_test, 2),
        batch_size=batch_size, shuffle=False),
        steps=(x_test.shape[0] // batch_size)+1, workers=4)

    indices = [np.random.choice(range(len(x_test))) 
               for i in range(num_predictions)]
    x_test,  y_test_list = subclass_np(x_test_dict, y_test_dict)
    cifar_grid_c(x_test,to_categorical(y_test, 2), indices,6, predictions=to_categorical(np.argmax(predict_gen,1),2))